In [323]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
import matplotlib.pyplot as plt
from imblearn.under_sampling import RandomUnderSampler
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_auc_score
import matplotlib.pyplot as plt
import seaborn as sns

output_file_path = 'output.txt'
# Загрузка данных
file_path = 'DataSets/dataframe_benign_portscan.csv'  # Укажите путь к вашему файлу
df = pd.read_csv(file_path)

# Просмотр первых строк для проверки
df.columns = [col.lstrip() for col in df.columns] # delete a space before name of the columns

# Проверка на наличие пропущенных значений
# print(df.isnull().sum())


# with open(output_file_path, 'w') as file:
#     file.write(df.head().to_string(index=False))



In [324]:
# Преобразование текстовых меток в числовые (если требуется)
# Например, если у вас есть столбец 'Label' с текстовыми метками
df['Label'] = np.where(df['Label']=='PortScan', 1, 0)


unique_labels = df['Label'].unique()
print("Уникальные метки в столбце 'Label':", unique_labels)
print("Количество NaN в столбце 'Label' после преобразования:", df['Label'].isnull().sum())


Уникальные метки в столбце 'Label': [0 1]
Количество NaN в столбце 'Label' после преобразования: 0


In [325]:
# Подсчет количества записей для каждого класса
class_counts = df['Label'].value_counts()
print("Количество записей для каждого класса:\n", class_counts)

# class_counts.plot(kind='bar')
# plt.title('Распределение классов')
# plt.xlabel('Класс')
# plt.ylabel('Количество записей')
# plt.show()


Количество записей для каждого класса:
 Label
1    158804
0    127292
Name: count, dtype: int64


In [326]:
# Выбрасываем столбец 'Label', axis=1 указывает, что мы выбрасываем столбец, а не строку
# X - это матрица признаков которых 0-68
# columns_to_exclude = ['Label', 'PSH Flag Count', 'Fwd Packet Length Max', 'Average Packet Size', 'Total Length of Fwd Packets', 'Subflow Fwd Bytes', 'Fwd Packet Length Mean', 'Avg Fwd Segment Size', 'Packet Length Mean', 'Subflow Fwd Packets', 'Fwd Packets/s', 'Init_Win_bytes_forward', 'Bwd Packet Length Mean', 'Bwd Packet Length Min']
columns_to_exclude = ['Label']
X = df.drop(columns_to_exclude, axis=1)

# y это значения label для каждого Label
y = df['Label']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.7, random_state=42)

# print(len(X_test))
# unique_labels = y_test.value_counts()
# print(unique_labels)

In [327]:
under_sampler = RandomUnderSampler(random_state=42)
X_resampled, y_resampled = under_sampler.fit_resample(X_train, y_train)

unique_labels = y_resampled.unique()
print(unique_labels)

# Подсчет количества записей для каждого класса
# print("Количество записей для каждого класса после undersampling:\n", y_resampled.value_counts())
# print(X_resampled.head())


[0 1]


In [331]:
# Обучение модели (например, RandomForest)
clf = RandomForestClassifier(n_estimators=1, random_state=42)
clf.fit(X_train, y_train)

        Destination Port  Flow Duration  Total Fwd Packets  \
130523              8701             44                  1   
149503              8086             25                  1   
156730             49176             67                  1   
258795               443      117429047                 13   
184779              9876             44                  1   

        Total Backward Packets  Total Length of Fwd Packets  \
130523                       1                            2   
149503                       1                            0   
156730                       1                            0   
258795                      11                          614   
184779                       1                            0   

        Total Length of Bwd Packets  Fwd Packet Length Max  \
130523                            6                      2   
149503                            6                      0   
156730                            6                      0   


RandomForestClassifier(n_estimators=1, random_state=42)

In [332]:
with open(output_file_path, 'w') as file:
    file.write(y_train.head().to_string(index=False))


#X_train
#y_train

In [329]:
# Предсказание на тестовых данных
y_pred = clf.predict(X_test)

# Расчет точности
accuracy = accuracy_score(y_test, y_pred)
print("Точность:", accuracy)

Точность: 0.8866003887187661


In [330]:
# Расчет ROC AUC
roc_auc = roc_auc_score(y_test, clf.predict_proba(X_test)[:, 1])
print("ROC AUC:", roc_auc)

ROC AUC: 0.8726905645713369
